Experiment: Averaging together last 4 attention distribution
------
averages together the last four attention distributions; using only one attention head.

In [1]:
import sys; sys.path.append('../..')
import torch
from bias_classification import prepare_model 
from src.utils import *

%load_ext autoreload
%autoreload 2

In [2]:
params = prepare_model.intialize_params("experiment_params.json")
dataset = prepare_model.initialize_dataset(params.intermediary_task)
attention_dataloader = dataset.return_dataloader()
attention_experiment = prepare_model.initialize_attention_experiment(params.intermediary_task)

06/03/2019 14:16:01 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
500it [00:00, 7540.81it/s]
06/03/2019 14:16:02 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../../../tasks/bias_classification/results/cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
06/03/2019 14:16:02 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../../../tasks/bias_classification/results/cache/9c41111e2de8

In [ ]:
# Run to print out the current key names in the dataset
dataset.get_key_names()

In [3]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [8]:
# Extracting indices of bias labels 
intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = [label.flatten().tolist().index(1) for label in intermediary_labels]
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)
# Summing together bias distributions
num_attention_layers = len(params.intermediary_task['attention']['layers'])
avg_dist = [dist/num_attention_layers for dist in sum_attention_dist(biased_tok_dist)]

In [9]:
print(avg_dist)

[tensor([9.3869e-02, 1.2798e-05, 5.4009e-06, 6.6591e-05, 7.5565e-05, 5.5644e-07,
        5.1555e-07, 3.4680e-06, 8.3441e-04, 1.8114e-03, 1.7443e-03, 5.1296e-01,
        2.3163e-04, 1.9506e-04, 6.6091e-04, 1.4891e-04, 3.4759e-05, 4.4659e-06,
        1.3219e-05, 3.8839e-06, 1.7966e-06, 1.3195e-05, 5.8550e-04, 4.1465e-03,
        2.2367e-03, 9.7067e-04, 6.5113e-04, 9.2916e-05, 2.5548e-04, 1.5163e-04,
        6.3845e-05, 5.2185e-05, 1.5320e-06, 4.5888e-05, 8.6990e-05, 3.6899e-03,
        2.6175e-02, 2.5727e-02, 2.1198e-02, 4.2316e-02, 5.7997e-02, 3.9715e-02,
        7.8523e-03, 7.4056e-04, 1.2492e-04, 5.2031e-05, 2.2407e-04, 2.4193e-04,
        2.7985e-04, 1.0516e-03, 1.3325e-03, 1.6044e-03, 1.0165e-03, 8.4160e-04,
        2.2833e-04, 2.3829e-04, 2.2137e-05, 3.7834e-05, 5.5707e-05, 7.4050e-05,
        8.5386e-07, 2.0669e-06, 4.5003e-05, 2.9532e-03, 1.9324e-02, 2.6630e-02,
        3.5099e-02, 3.4880e-02, 1.0993e-02, 6.1030e-03, 6.7330e-04, 1.0990e-04,
        1.6003e-04, 5.8813e-05, 7.0148e

In [10]:
windowed_data, removed_indices = window_attention_dist(avg_dist, bias_indices, window_size=params.final_task['window_size'])
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(windowed_data), "attention_dist")
dataset.shuffle_data()

Run the cell below to reset params 
-----

In [13]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [14]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 11, 'hidden_dim': 3, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [15]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [16]:
statistics

{'auc': [(0.5870157757885095, 0.805444877252462), 0.7149701973519355],
 'accuracy': [(0.5752906976744185, 0.7558139534883721), 0.6793023255813954]}

New run 

In [35]:
#reloading classification experiment with new experiments
params = prepare_model.intialize_params("experiment_params.json") 
classification_experiment = prepare_model.initialize_classification_experiment(params.final_task)

In [36]:
print(params.final_task)

{'model': 'shallow_nn', 'window_size': 5, 'input_dim': 11, 'hidden_dim': 3, 'output_dim': 1, 'data_split': {'train_split': 0.7, 'eval_split': 0.3, 'test_split': 0}, 'training_params': {'optimizer': 'adam', 'loss': 'bce_with_logits', 'num_epochs': 200, 'batch_size': 32, 'lr': 0.01}}


Boostrapping procedure 

In [ ]:
statistics = run_boostrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [22]:
statistics

{'auc': [(0.49685111400227683, 0.8255225075881711), 0.7107909485455589],
 'accuracy': [(0.5869186046511627, 0.7502906976744186), 0.6730232558139536]}

## No bootstrapping -- cannot calculate 95% confidence intervals 

In [23]:
data_split = params.final_task['data_split']
batch_size = params.final_task['training_params']['batch_size']
train_dataloader, eval_dataloader, test_dataloader = dataset.split_train_eval_test(**data_split, batch_size=batch_size)

In [24]:
losses, evaluations = classification_experiment.train_model(train_dataloader, eval_dataloader, input_key='attention_dist', label_key='bias_label', threshold=0.42)

In [25]:
avg_losses = [average_data(epoch_losses) for epoch_losses in losses]
avg_predictions = [average_data(epoch_evaluations) for epoch_evaluations in evaluations]

In [26]:
min_loss, max_loss, avg_loss = get_statistics(avg_losses, "loss")
print(min_loss)

0.5602870774269103


In [27]:
min_auc, max_auc, avg_auc = get_statistics(avg_predictions, "auc")

In [28]:
print(max_auc)

0.6637993874070985


In [31]:
print(min_auc)

0.6379312454511352
